In [1]:
import LinearAlgebra as la
using BenchmarkTools: @btime
include("hoSystem.jl");

In [2]:
l_0 = 10  # number of basis functions
ω = 0.25  # strength of harmonic oscillator potential
a = 0.25  # shielding term in coloumb interaction
grid = [x for x in range(-10, stop = 10, length = 2001)]
n = 2     # number of particles

@time system = System(n, HOBasis(l_0, ω, a), grid);

  1.231972 seconds (5.05 M allocations: 298.628 MiB, 4.89% gc time, 8.33% compilation time)


In [3]:
# Stand-in for HF solution
C = rand(system.l, system.l);

In [4]:
function slow_ho(results, x, basis)
    for (j, ψ) in enumerate(basis.funcs)
        results[j] = compute(x, ψ)
    end
    
    return results
end

slow_ho (generic function with 1 method)

In [5]:
N = 50
x = 10
hos = zeros(N);
hermites = zeros(N);

In [6]:
basis = HOBasis(N, ω, a)
slow = zeros(N);
slow = slow_ho(slow, x, basis);

In [11]:
fast = zeros(N);
fast = fast_ho(fast, hermites, x, ω);

In [12]:
fast_prob = zeros(N);
fast_prob = fast_ho_prob(fast_prob, hermites, x, ω);

In [9]:
function fast_ho(hos, hermites, x, ω)
    x = √ω * x

    hermites[1] = 1.0
    hermites[2] = 2x

    ho_fac = (ω / π)^0.25 * exp(-x^2 / 2)
    hos[1] = ho_fac * hermites[1]
    ho_fac *= 1 / √2
    hos[2] = ho_fac * hermites[2]

    for n in 3:length(hos)
        hermites[n] = 2x * hermites[n-1] - 2(n - 2) * hermites[n-2]

        ho_fac *= 1 / sqrt( 2(n - 1) )
        hos[n] = ho_fac * hermites[n]
    end
    
    return hos
end

fast_ho (generic function with 1 method)

In [10]:
function fast_ho_prob(hos, hermites, x, ω)
    x = √ω * x
    ho_fac = (ω / π)^0.25 * exp(-x^2 / 2)
    x = √2 * x
    
    hermites[1] = 1.0
    hermites[2] = x

    hos[1] = ho_fac * hermites[1]
    hos[2] = ho_fac * hermites[2]

    for n in 3:length(hos)
        hermites[n] = x * hermites[n-1] - (n - 2) * hermites[n-2]

        ho_fac *= 1 / sqrt(n - 1)
        hos[n] = ho_fac * hermites[n]
    end
    
    return hos
end

fast_ho_prob (generic function with 1 method)